In [226]:
import json
from pathlib import Path

TSV_PATH = Path("/Users/joregan/Playing/hsi/annotations/word_annotations/main")
JSON_PATH = Path("/Users/joregan/Playing/hsi/annotations/final_resolved")

In [227]:
old_json = {}
tsv_cache = {}

In [228]:
def slurp(filename):
    with open(filename) as f:
        segment = json.load(f)
    return segment

In [246]:
def get_topic_context(segment, size=None, keep_topic=True):
    rec_id = segment["recording_id"]
    orig_seg_id = segment["segment_id"]
    if not rec_id in old_json:
        with open(JSON_PATH / f"{rec_id}.json") as inf:
            old_json[rec_id] = json.load(inf)
    original = old_json[rec_id]
    orig_keys = list(original.keys())
    orig_keys.sort(key=lambda x: int(x))
    orig_topic = original[orig_seg_id]["high_level"]["current_topic"]

    index = orig_keys.index(orig_seg_id)
    if size is None:
        start = 0
    else:
        start = index - size
    ctx_range = orig_keys[start:index]

    if size is not None and len(ctx_range) < size:
        if int(orig_seg_id) <= size:
            pass
        else:
            print(f"Warning: size of {size} cannot be satisfied: {ctx_range}")
    
    topics = [original[x]["high_level"]["current_topic"] for x in ctx_range]

    tmp = []
    for p in zip(ctx_range, topics):
        if not keep_topic:
            tmp.append(original[p[0]]["snippet"])
        elif keep_topic and p[1] == orig_topic:
            tmp.append(original[p[0]]["snippet"])
        else:
            tmp.append(None)
    return " ".join([x for x in tmp if x is not None])

In [230]:
def get_time_context(segment, ctx_time = 5.0):
    rec_id = segment["recording_id"]
    start = segment["timing"]["utterance_start"]

    if not rec_id in tsv_cache:
        with open(TSV_PATH / f"{rec_id}_main.tsv") as inf:
            lines = []
            for line in inf.readlines():
                line = line.strip()
                if "\t" in line:
                    lines.append(line.split("\t"))
            tsv_cache[rec_id] = lines

    tsv_times = tsv_cache[rec_id]
    extract = []
    for time in tsv_times:
        s = float(time[0])
        e = float(time[1])
        if s >= (start - ctx_time) and (e < start):
            extract.append(time[2])
    return " ".join(extract)

In [236]:
seg = slurp("/Users/joregan/Playing/hsi/annotations/meta/hsi_7_0719_227_003_68_000_meta.json")
txt_ctx = get_topic_context(seg, 5)
aud_ctx = get_time_context(seg, 30.0)

print(txt_ctx)
print(aud_ctx)

hsi_7_0719_227_003 68
Yeah. Ah. That one, it seems like it doesn't have enough light because it's a little bit sad. So I would just put them really, really next to the windows and where the sun comes because you're not facing south here, you're facing east. So yeah, I would put it like next to the do- next to the door and really keep it there in the mornings because um I think it really needs some light.
The plant in the middle of the couches? Yeah. Ah. That one, it seems like it doesn't have enough light because it's a little bit sad. So I would just put them really, really next to the windows and where the sun comes because you're not facing south here, you're facing east. So yeah, I would put it like next to the do- next to the door and really keep it there in the mornings because um I think it really needs some light.


In [249]:
META_PATH = Path("/Users/joregan/Playing/hsi/annotations/meta")

for file in META_PATH.glob("*.json"):
    seg = slurp(str(file))
    txt_ctx = get_topic_context(seg, 5)
    aud_ctx = get_time_context(seg, 20.0)

    print("TEXT", txt_ctx)
    print("AUDIO", aud_ctx)
    print("UTTERANCE", seg["utterance"])
    print()

TEXT See about the mat. Did you go with the vacuum cleaner on the mat or not?
AUDIO You didn't get the br- the brushes. It was n- next to the vacuum cleaner. See about the mat. Did you go with the vacuum cleaner on the mat or not?
UTTERANCE Paper!

TEXT Ah you mean the brown one or the black one?
AUDIO uh of cohesion with the rest of the room. So, for example, with the couches and the grey and, you know, everything is uh... a little bit balanced, let's say. Ah you mean the brown one or the black one?
UTTERANCE Okay, yeah, that is a really, really eh nice sculpture that a friend of mine gifted me uh on a trip to Africa.

TEXT Uhm. Could maybe... I'm thinking but if yeah, cuz if we would put the couch and chair there facing the fireplace, the desk would be turned that way. So it would be more of a like, Area for riding and stuff over there.
AUDIO Could maybe... I'm thinking but if yeah, cuz if we would put the couch and chair there facing the fireplace, the desk would be turned that way.

In [ ]:
# totally unrelated code to extract a list of tuples from a vlm
def get_answer(text):
    flat = text.replace(" ", "")
    if ")]" in flat:
        return eval(flat[flat.find("[("):flat.rfind(")]")+2])
    else:
        return eval(flat[flat.find("[("):flat.rfind(")")+1] + "]")